In [1]:
import sys
sys.path.insert(0,'../')
from file_tools import *
from request_tools import *
from parse_tools import *
import time
from tqdm import tqdm
import argparse

# Filter Runtime Warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [2]:
def __normalize_league_gamelogs_stats_minmax__(df):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler((-1,1))
    df_norm = df.copy()
    df_norm.loc[:,:] = scaler.fit_transform(df_norm)
    return df_norm

def __normalize_league_gamelogs_stats_standard__(df):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df_norm = df.copy()
    df_norm.loc[:,:] = scaler.fit_transform(df_norm)
    return df_norm

def __normalize_league_gamelogs_stats_robust__(df):
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    df_norm = df.copy()
    df_norm.loc[:,:] = scaler.fit_transform(df_norm)
    return df_norm

def __normalize_league_gamelogs_stats_ranking__(df):
    return df.copy().rank(axis=0,method='min',ascending=False)

SRC_DIR = './data-aggregated/'
TGT_DIR = './data-normalized/'


In [12]:
LG_SS_HTML_DICT = {}
if file_exists(f'{SRC_DIR}/league_seasons_html.txt'):
    LG_SS_HTML_DICT_STR = load_file(f'{SRC_DIR}/league_seasons_html.txt')
    LG_SS_HTML_DICT = ast.literal_eval(LG_SS_HTML_DICT_STR)
else:
    print('league_seasons_html.txt not found. Please scrape it first.')

TGL_STATS_TYPE = 'tgl_basic/stats_hm_cumu_avg'
LG_SS_HTML, LG_TM_HTML_LIST = list(LG_SS_HTML_DICT.items())[1]
LG_TGL_STATS_PER_TM_LIST = []
for TM_SS_HTML_ID in LG_TM_HTML_LIST:
    TM_SS_DIR = parse_team_season_id(TM_SS_HTML_ID)['body']
    TGL_STATS_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/{TGL_STATS_TYPE}.csv',header=[0,1],index_col=[0,1])
    TGL_STATS_DF.index.names = ['index','Team_id']
    LG_TGL_STATS_PER_TM_LIST.append(TGL_STATS_DF)

for RD_INDEX in range(len(LG_TGL_STATS_PER_TM_LIST[0])):
    LG_TGL_STATS_PER_RD_DF = []
    for TGL_STATS_TM in LG_TGL_STATS_PER_TM_LIST:
        if RD_INDEX >= len(TGL_STATS_TM):
            continue
        RD_DF = TGL_STATS_TM.iloc[RD_INDEX].to_frame().T
        RD_DF.index.names = ['index','Team_id']    
        LG_TGL_STATS_PER_RD_DF.append(RD_DF)
    LG_TGL_STATS_PER_RD_DF = pd.concat(LG_TGL_STATS_PER_RD_DF,axis=0)
    LG_TGL_STATS_PER_RD_NORM_MNMX_DF  = __normalize_league_gamelogs_stats_minmax__(LG_TGL_STATS_PER_RD_DF)

    # LG_TGL_STATS_PER_RD_NORM_STD_DF   = __normalize_league_gamelogs_stats_standard__(LG_TGL_STATS_PER_RD_DF)
    # LG_TGL_STATS_PER_RD_NORM_RBST_DF  = __normalize_league_gamelogs_stats_robust__(LG_TGL_STATS_PER_RD_DF)
    # LG_TGL_STATS_PER_RD_NORM_RANK_DF  = __normalize_league_gamelogs_stats_ranking__(LG_TGL_STATS_PER_RD_DF)
LG_TGL_STATS_PER_RD_DF

#     # Save each df
#     RD_NM = f'{RD_INDEX:02d}'
#     LG_TGL_STATS_PER_RD_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/unnorm/{RD_NM}.csv')
#     LG_TGL_STATS_PER_RD_NORM_MNMX_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_minmax/{RD_NM}.csv')
#     LG_TGL_STATS_PER_RD_NORM_STD_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_standard/{RD_NM}.csv')
#     LG_TGL_STATS_PER_RD_NORM_RBST_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_robust/{RD_NM}.csv')
#     LG_TGL_STATS_PER_RD_NORM_RANK_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_ranking/{RD_NM}.csv')
# for RD_INDEX in range(len(LG_TGL_STATS_PER_TM_LIST[0])):
#     LG_TGL_STATS_PER_RD_DF = []
#     for TGL_STATS_TM in LG_TGL_STATS_PER_TM_LIST:

Result                               Team   
                                 W/L          Tm         Opp         FG   
index Team_id                                                             
80    /teams/ATL/2023.html  0.585366  119.634146  117.390244  45.341463  \
81    /teams/BOS/2023.html  0.780488  120.512195  110.463415  42.731707   
      /teams/BRK/2023.html  0.560976  113.024390  110.048780  41.463415   
      /teams/CHI/2023.html  0.536585  114.048780  111.170732  42.682927   
80    /teams/CHO/2023.html  0.317073  109.243902  116.146341  40.365854   
81    /teams/CLE/2023.html  0.756098  113.170732  104.975610  41.756098   
      /teams/DAL/2023.html  0.560976  115.365854  112.682927  40.634146   
      /teams/DEN/2023.html  0.829268  119.390244  109.609756  44.658537   
79    /teams/DET/2023.html  0.219512  112.317073  118.512195  40.292683   
      /teams/GSW/2023.html  0.804878  119.707317  111.731707  43.634146   
      /teams/HOU/2023.html  0.341463  110.756098  115.146341  40.390244   
80    /teams/IND/2023.html  0.487805  118.926829  119.731707  42.951220   
      /teams/LAC/2023.html  0.560976  112.731707  110.585366  41.024390   
81    /teams/LAL/2023.html  0.560976  117.024390  113.804878  42.975610   
78    /teams/MEM/2023.html  0.853659  119.780488  109.195122  45.146341   
81    /teams/MIA/2023.html  0.658537  111.439024  110.243902  39.731707   
80    /teams/MIL/2023.html  0.780488  118.829268  112.487805  43.414634   
81    /teams/MIN/2023.html  0.536585  115.926829  115.024390  42.731707   
80    /teams/NOP/2023.html  0.658537  114.926829  109.926829  41.853659   
81    /teams/NYK/2023.html  0.560976  117.268293  113.000000  42.585366   
      /teams/OKC/2023.html  0.585366  120.829268  115.634146  44.634146   
79    /teams/ORL/2023.html  0.487805  114.317073  114.731707  41.487805   
      /teams/PHI/2023.html  0.707317  114.195122  109.536585  40.146341   
81    /teams/PHO/2023.html  0.682927  114.097561  109.243902  41.804878   
      /teams/POR/2023.html  0.414634  115.609756  119.317073  40.829268   
80    /teams/SAC/2023.html  0.560976  123.365854  120.097561  44.536585   
      /teams/SAS/2023.html  0.341463  115.073171  121.414634  44.097561   
81    /teams/TOR/2023.html  0.658537  114.365854  109.731707  42.146341   
80    /teams/UTA/2023.html  0.560976  117.951220  116.878049  41.707317   
81    /teams/WAS/2023.html  0.463415  114.243902  114.609756  42.439024   

                                                                        
                                  FGA       FG%         3P        3PA   
index Team_id                                                           
80    /teams/ATL/2023.html  94.219512  0.482561  10.707317  30.853659  \
81    /teams/BOS/2023.html  88.560976  0.484415  16.243902  42.902439   
      /teams/BRK/2023.html  85.463415  0.486073  13.073171  34.219512   
      /teams/CHI/2023.html  86.243902  0.496220  10.268293  28.121951   
80    /teams/CHO/2023.html  89.121951  0.455341  11.073171  32.658537   
81    /teams/CLE/2023.html  84.902439  0.494220  11.853659  32.390244   
      /teams/DAL/2023.html  85.536585  0.477293  14.926829  40.902439   
      /teams/DEN/2023.html  86.902439  0.514805  12.365854  31.682927   
79    /teams/DET/2023.html  87.121951  0.463439  11.780488  32.780488   
      /teams/GSW/2023.html  90.780488  0.481976  17.073171  43.682927   
      /teams/HOU/2023.html  88.341463  0.457415  10.487805  31.951220   
80    /teams/IND/2023.html  90.682927  0.474634  13.951220  37.926829   
      /teams/LAC/2023.html  85.341463  0.481463  12.609756  32.926829   
81    /teams/LAL/2023.html  89.073171  0.484537  11.243902  31.756098   
78    /teams/MEM/2023.html  93.073171  0.486268  12.292683  34.804878   
81    /teams/MIA/2023.html  84.756098  0.469561  11.951220  33.243902   
80    /teams/MIL/2023.html  90.609756  0.481049  14.926829  39.829268   
81    /teams/MIN/2023.html  88.146341  0.487000  12.341463  34.780488   
80    /teams/NOP/2023

In [21]:
LG_SS_HTML_DICT = {}
if file_exists(f'{SRC_DIR}/league_seasons_html.txt'):
    LG_SS_HTML_DICT_STR = load_file(f'{SRC_DIR}/league_seasons_html.txt')
    LG_SS_HTML_DICT = ast.literal_eval(LG_SS_HTML_DICT_STR)
else:
    print('league_seasons_html.txt not found. Please scrape it first.')

TGL_STATS_TYPE = 'tgl_basic/facts_venue_rest_days'
LG_SS_HTML, LG_TM_HTML_LIST = list(LG_SS_HTML_DICT.items())[1]
LG_TGL_STATS_PER_TM_LIST = []
for TM_SS_HTML_ID in LG_TM_HTML_LIST:
    TM_SS_DIR = parse_team_season_id(TM_SS_HTML_ID)['body']
    TGL_STATS_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/{TGL_STATS_TYPE}.csv',header=[0,1],index_col=[0,1])
    TGL_STATS_DF.index.names = ['index','Team_id']
    LG_TGL_STATS_PER_TM_LIST.append(TGL_STATS_DF)

for RD_INDEX in range(len(LG_TGL_STATS_PER_TM_LIST[0])):
    LG_TGL_STATS_PER_RD_DF = []
    for TGL_STATS_TM in LG_TGL_STATS_PER_TM_LIST:
        if RD_INDEX >= len(TGL_STATS_TM):
            continue
        RD_DF = TGL_STATS_TM.iloc[RD_INDEX].to_frame().T
        RD_DF.index.names = ['index','Team_id']    
        LG_TGL_STATS_PER_RD_DF.append(RD_DF)
    LG_TGL_STATS_PER_RD_DF = pd.concat(LG_TGL_STATS_PER_RD_DF,axis=0)

LG_TGL_STATS_PER_RD_DF

Match                                            
                             H/A Venue_Strk_Cnt Days_Btw_2GM Days_Btw_3GM   
index Team_id                                                               
81    /teams/ATL/2023.html   0.0            0.0          2.0          4.0  \
      /teams/BOS/2023.html   1.0            2.0          2.0          4.0   
      /teams/BRK/2023.html   1.0            1.0          2.0          4.0   
      /teams/CHI/2023.html   1.0            0.0          2.0          4.0   
      /teams/CHO/2023.html   0.0            0.0          2.0          5.0   
      /teams/CLE/2023.html   1.0            0.0          3.0          5.0   
      /teams/DAL/2023.html   1.0            2.0          2.0          4.0   
      /teams/DEN/2023.html   1.0            0.0          1.0          3.0   
      /teams/DET/2023.html   0.0            1.0          2.0          4.0   
      /teams/GSW/2023.html   0.0            1.0          2.0          5.0   
      /teams/HOU/2023.html   0.0            1.0          2.0          5.0   
      /teams/IND/2023.html   0.0            0.0          2.0          4.0   
      /teams/LAC/2023.html   0.0            0.0          1.0          4.0   
      /teams/LAL/2023.html   1.0            1.0          2.0          4.0   
      /teams/MEM/2023.html   0.0            2.0          2.0          4.0   
      /teams/MIA/2023.html   1.0            0.0          2.0          3.0   
      /teams/MIL/2023.html   0.0            0.0          2.0          4.0   
      /teams/MIN/2023.html   1.0            0.0          1.0          5.0   
      /teams/NOP/2023.html   0.0            0.0          2.0          4.0   
      /teams/NYK/2023.html   1.0            0.0          2.0          4.0   
      /teams/OKC/2023.html   1.0            0.0          3.0          5.0   
      /teams/ORL/2023.html   0.0            1.0          2.0          3.0   
      /teams/PHI/2023.html   0.0            1.0          2.0          3.0   
      /teams/PHO/2023.html   1.0            0.0          2.0          3.0   
      /teams/POR/2023.html   1.0            0.0          1.0          3.0   
      /teams/SAC/2023.html   0.0            0.0          2.0          4.0   
      /teams/SAS/2023.html   0.0            0.0          1.0          3.0   
      /teams/TOR/2023.html   1.0            0.0          2.0          4.0   
      /teams/UTA/2023.html   0.0            0.0          1.0          3.0   
      /teams/WAS/2023.html   1.0            1.0          2.0          4.0   

                                         
                           Days_Btw_4GM  
index Team_id                            
81    /teams/ATL/2023.html          5.0  
      /teams/BOS/2023.html          5.0  
      /teams/BRK/2023.html          5.0  
      /teams/CHI/2023.html          5.0  
      /teams/CHO/2023.html          7.0  
      /teams/CLE/2023.html          7.0  
      /teams/DAL/2023.html          7.0  
      /teams/DEN/2023.html          5.0  
      /teams/DET/2023.html          5.0  
      /teams/GSW/2023.html          7.0  
      /teams/HOU/2023.html          7.0  
      /teams/IND/2023.html          7.0  
      /teams/LAC/2023.html          8.0  
      /teams/LAL/2023.html          5.0  
      /teams/MEM/2023.html          5.0  
      /teams/MIA/2023.html          5.0  
      /teams/MIL/2023.html          5.0  
      /teams/MIN/2023.html          7.0  
      /teams/NOP/2023.html          5.0  
      /teams/NYK/2023.html          7.0  
      /teams/OKC/2023.html          7.0  
      /teams/ORL/2023.html          5.0  
      /teams/PHI/2023.html          5.0  
      /teams/PHO/2023.html          5.0  
      /teams/POR/2023.html          5.0  
      /teams/SAC/2023.html          5.0  
      /teams/SAS/2023.html          5.0  
      /teams/TOR/2023.html          5.0  
      /teams/UTA/2023.html          5.0  
      /teams/WAS/2023.html          5.0

In [ ]:

# def normalize_gamelogs_stats_regular_season():
"""
Description:
    Normalize gamelog stats relative to the current season, organized by round
Summary:

Usage:
    python3 04_normalize_script.py -s ../data-aggregated/ -t ../data-normalized/

"""
# Load list of league seasons
LG_SS_HTML_DICT = {}
if file_exists(f'{SRC_DIR}/league_seasons_html.txt'):
    LG_SS_HTML_DICT_STR = load_file(f'{SRC_DIR}/league_seasons_html.txt')
    LG_SS_HTML_DICT = ast.literal_eval(LG_SS_HTML_DICT_STR)
else:
    print('league_seasons_html.txt not found. Please scrape it first.')
    # return

# For every league_season, get all team_seasons belonging to the league
TQDM_LG_SS_HTML_DICT = tqdm(LG_SS_HTML_DICT.items(),position=0, leave=True,ncols=125)
for LG_SS_HTML, LG_TM_HTML_LIST in TQDM_LG_SS_HTML_DICT:
    TQDM_LG_SS_HTML_DICT.set_description(f'{LG_SS_HTML}')
    LG_SS_DIR = parse_league_id(LG_SS_HTML)['body']
    # For each stats type
    for TGL_STATS_TYPE in [ 
        'tgl_basic/stats_cumu_avg', 'tgl_basic/stats_roll_04_avg','tgl_basic/stats_roll_08_avg', 
        'tgl_basic/stats_roll_12_avg', 'tgl_basic/stats_roll_16_avg', 'tgl_basic/stats_roll_20_avg',
        'tgl_basic/stats_hm_cumu_avg', 'tgl_basic/stats_hm_roll_04_avg','tgl_basic/stats_hm_roll_08_avg',
        'tgl_basic/stats_hm_roll_12_avg', 'tgl_basic/stats_hm_roll_16_avg', 'tgl_basic/stats_hm_roll_20_avg',
        'tgl_basic/stats_aw_cumu_avg',  'tgl_basic/stats_aw_roll_04_avg','tgl_basic/stats_aw_roll_08_avg',
        'tgl_basic/stats_aw_roll_12_avg', 'tgl_basic/stats_aw_roll_16_avg', 'tgl_basic/stats_aw_roll_20_avg',
        # 'tgl_basic/stats_h2h_cumu_avg',
        'tgl_advanced/stats_cumu_avg', 'tgl_advanced/stats_roll_04_avg','tgl_advanced/stats_roll_08_avg',
        'tgl_advanced/stats_roll_12_avg', 'tgl_advanced/stats_roll_16_avg', 'tgl_advanced/stats_roll_20_avg',
        'tgl_advanced/stats_hm_cumu_avg', 'tgl_advanced/stats_hm_roll_04_avg','tgl_advanced/stats_hm_roll_08_avg',
        'tgl_advanced/stats_hm_roll_12_avg', 'tgl_advanced/stats_hm_roll_16_avg', 'tgl_advanced/stats_hm_roll_20_avg',
        'tgl_advanced/stats_aw_cumu_avg',  'tgl_advanced/stats_aw_roll_04_avg','tgl_advanced/stats_aw_roll_08_avg',
        'tgl_advanced/stats_aw_roll_12_avg', 'tgl_advanced/stats_aw_roll_16_avg', 'tgl_advanced/stats_aw_roll_20_avg',
        # 'tgl_advanced/stats_h2h_cumu_avg',
    ]:
        # Get the corresponding season gamelogs stats for each team belonging to the league
        LG_TGL_STATS_PER_TM_LIST = []
        for TM_SS_HTML_ID in LG_TM_HTML_LIST:
            TM_SS_DIR = parse_team_season_id(TM_SS_HTML_ID)['body']
            TGL_STATS_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/{TGL_STATS_TYPE}.csv',header=[0,1],index_col=[0,1])
            LG_TGL_STATS_PER_TM_LIST.append(TGL_STATS_DF)

        # Create a table for each round in the league season
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/unnorm')
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_minmax')
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_standard')
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_robust')
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_ranking')
        for RD_INDEX in range(len(LG_TGL_STATS_PER_TM_LIST[0])):
            LG_TGL_STATS_PER_RD_DF = []
            for TGL_STATS_TM in LG_TGL_STATS_PER_TM_LIST:
                if RD_INDEX >= len(TGL_STATS_TM):
                    continue
                RD_DF = TGL_STATS_TM.iloc[RD_INDEX].to_frame().T
                RD_DF.index.names = ['index','Team_id']    
                LG_TGL_STATS_PER_RD_DF.append(RD_DF)
            LG_TGL_STATS_PER_RD_DF = pd.concat(LG_TGL_STATS_PER_RD_DF,axis=0)
            LG_TGL_STATS_PER_RD_NORM_MNMX_DF  = __normalize_league_gamelogs_stats_minmax__(LG_TGL_STATS_PER_RD_DF)
            LG_TGL_STATS_PER_RD_NORM_STD_DF   = __normalize_league_gamelogs_stats_standard__(LG_TGL_STATS_PER_RD_DF)
            LG_TGL_STATS_PER_RD_NORM_RBST_DF  = __normalize_league_gamelogs_stats_robust__(LG_TGL_STATS_PER_RD_DF)
            LG_TGL_STATS_PER_RD_NORM_RANK_DF  = __normalize_league_gamelogs_stats_ranking__(LG_TGL_STATS_PER_RD_DF)

            # Save each df
            RD_NM = f'{RD_INDEX:02d}'
            LG_TGL_STATS_PER_RD_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/unnorm/{RD_NM}.csv')
            LG_TGL_STATS_PER_RD_NORM_MNMX_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_minmax/{RD_NM}.csv')
            LG_TGL_STATS_PER_RD_NORM_STD_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_standard/{RD_NM}.csv')
            LG_TGL_STATS_PER_RD_NORM_RBST_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_robust/{RD_NM}.csv')
            LG_TGL_STATS_PER_RD_NORM_RANK_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/norm_ranking/{RD_NM}.csv')

    for TGL_STATS_TYPE in [ 
        'tgl_basic/facts_gm_results', 'tgl_basic/facts_venue_rest_days',
    ]:
        # Get the corresponding season gamelogs stats for each team belonging to the league
        LG_TGL_FACTS_PER_TM_LIST = []
        for TM_SS_HTML_ID in LG_TM_HTML_LIST:
            TM_SS_DIR = parse_team_season_id(TM_SS_HTML_ID)['body']
            TGL_STATS_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/{TGL_STATS_TYPE}.csv',header=[0],index_col=[0,1]) # header=[0] compared to header=[0,1]
            LG_TGL_FACTS_PER_TM_LIST.append(TGL_STATS_DF)

        # Create a table for each round in the league season
        make_directory(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/facts')
        for RD_INDEX in range(len(LG_TGL_FACTS_PER_TM_LIST[0])):
            LG_TGL_FACTS_PER_RD_DF = []
            for TGL_STATS_TM in LG_TGL_FACTS_PER_TM_LIST:
                if RD_INDEX >= len(TGL_STATS_TM):
                    continue
                RD_DF = TGL_STATS_TM.iloc[RD_INDEX].to_frame().T
                RD_DF.index.names = ['index','Team_id']    
                LG_TGL_FACTS_PER_RD_DF.append(RD_DF)
            LG_TGL_FACTS_PER_RD_DF = pd.concat(LG_TGL_FACTS_PER_RD_DF,axis=0)

            # Save each df
            RD_NM = f'{RD_INDEX:02d}'
            LG_TGL_FACTS_PER_RD_DF.to_csv(f'{TGT_DIR}/{LG_SS_DIR}/{TGL_STATS_TYPE}/facts/{RD_NM}.csv')




